### Importing packages and modules

In [64]:
# module for building the pyomo model
import pyomo.environ as pe
# module for solving the pyomo model
import pyomo.opt as po

### Create the model

In [65]:
model = pe.ConcreteModel()

Order to build the model:
1. Sets
1. Parameters
1. Variables
1. Objective function
1. Constraints

#### Sets

$i$: filas de la malla = {1,2,...,N_FIL}

$𝑗$: columnas de la malla = {1,2,...,N_COL}

$k$: conjunto de camiones disponibles = {1,2,...,NC}


In [66]:
N_FIL = 3  
N_COL = 3
MAX_STEPS = 2 * N_FIL * N_COL
# N_TRUCKS = 2

In [67]:
model.row = pe.Set(initialize=[i for i in range(N_FIL)])
model.column = pe.Set(initialize=[j for j in range(N_COL)])
# model.truck = pe.Set(initialize=[k for k in range(N_TRUCKS)])
model.step = pe.Set(initialize=[i for i in range(MAX_STEPS)])

#### Parameters

In [68]:
ND = 1 # Numero de depositos
T = 3 # Tiempo traslado nodo-nodo
model.time = pe.Param(initialize = T)

In [69]:
malla_vacia = {}

# Iterar sobre las filas y columnas de la matriz 10x10
for i in range(N_FIL):
    for j in range(N_COL):
        malla_vacia[(i, j)] = 0

$CP(i,j)$: Cantidad de basura que hay en la parada regular en (i,j)

In [70]:
# Total 10 kilos
dirt_regular_stops = malla_vacia
stop_position_1 = (2,1)
dirt_regular_stops[stop_position_1] = 7
stop_position_2 = (2,2)
dirt_regular_stops[stop_position_2] = 3

model.dirt_regular_stops = pe.Param(model.row, model.column, initialize = dirt_regular_stops)

$D(i,j)$: Depósito en (i,j)

In [71]:
deposit = malla_vacia
deposit_position = (0,0)
deposit[deposit_position] = 1

model.deposit = pe.Param(model.row, model.column, initialize = deposit)

$C_k$: capacidad del vehículo k

In [72]:
capacity = 10.0

model.capacity = pe.Param(initialize = capacity)

#### Variables



In [73]:
# Recorrido del camion, unos y ceros para ver por donde pasa
model.r = pe.Var(model.row, model.column, model.step, within = pe.Binary)

# Cantidad de basura del camion
model.b = pe.Var(model.step, within = pe.NonNegativeReals)

# Variable binaria que indica si toda la basura ha sido recogida
model.all_dirt_collected = pe.Var(model.step, within=pe.Binary)

# Numero de pasos que se recorre para volver al deposito
model.return_step = pe.Var(within=pe.NonNegativeIntegers, bounds=(0, MAX_STEPS))

# Marcar si la basura de una celda ha sido recogida en cada paso
model.is_collected = pe.Var(model.row, model.column, model.step, domain=pe.Binary)

#### Objective Function


In [74]:
# Función objetivo: Minimizar los pasos de vuelta , y teniendo en cuenta llegar lo antes posible a
# de vuelta al deposito, y recogiendo toda la basura, ya que al deposito solo se puede pasar dos
# veces y solo se puede pasar la segunda vez una vez recogida toda la basura

model.obj = pe.Objective(
    expr=model.return_step, 
    sense=pe.minimize
)

#### Constraints

In [75]:
# HE CAMBIADO LA RESTRICCIÓN 3 Y AÑADIDO LA 11

# Elimina restricciones previas
model.del_component('constraints')
model.constraints = pe.ConstraintList()

# 1. El camión empieza en el depósito
model.constraints.add(model.r[deposit_position[0], deposit_position[1], 0] == 1)

# 2. Restricción de movimiento Manhattan (movimientos solo a celdas adyacentes)
for s in model.step:
    for i in model.row:
        for j in model.column:
            if s > 0:
                model.constraints.add(
                    model.r[i, j, s] <= 
                    (model.r[i-1, j, s-1] if i > 0 else 0) +  # Arriba
                    (model.r[i+1, j, s-1] if i < N_FIL - 1 else 0) +  # Abajo
                    (model.r[i, j-1, s-1] if j > 0 else 0) +  # Izquierda
                    (model.r[i, j+1, s-1] if j < N_COL - 1 else 0)  # Derecha
                )

# 3. Restricción de una sola celda ocupada en cada paso de tiempo
for s in model.step:
    model.constraints.add(
        sum(model.r[i, j, s] for i in model.row for j in model.column) <= 1
    )

# 4. Restricción de capacidad del camión
for s in model.step:
    model.constraints.add(
        sum(model.dirt_regular_stops[i, j] * model.r[i, j, s] for i in model.row for j in model.column) <= model.capacity
    )

# 5. Restricción para actualizar la carga del camión al recoger basura
# Restringir que la carga solo se acumule la primera vez que se visita una celda de basura
for s in model.step:
    if s > 0:
        model.constraints.add(
            model.b[s] == model.b[s-1] + sum(
                model.dirt_regular_stops[i, j] * model.r[i, j, s] * (1 - model.is_collected[i, j, s-1])
                for i in model.row for j in model.column
            )
        )
    else:
        model.constraints.add(model.b[s] == 0)  # Inicialmente vacío

# 6. Restricción para garantizar que una celda de basura solo se marque como recolectada una vez
for s in model.step:
    for i in model.row:
        for j in model.column:
            if s > 0:
                # is_collected se activa solo si la celda es visitada
                model.constraints.add(
                    model.is_collected[i, j, s] >= model.r[i, j, s]
                )
                # Una vez activado, se mantiene en pasos posteriores
                model.constraints.add(
                    model.is_collected[i, j, s] >= model.is_collected[i, j, s-1]
                )

# 7. Activar 'all_dirt_collected' cuando toda la basura ha sido recogida
BIG_M = 10000  
small_m = 1*10**-4

# Puede ser 0 o 1 si es 0
# Es 0 si es != 0
for s in model.step:
    model.constraints.add(
        BIG_M * (1 - model.all_dirt_collected[s]) >= sum(model.dirt_regular_stops[i, j] for i in model.row for j in model.column) - model.b[s]
    )

# Es 1 si es 0
# Puede ser 0 o 1 si es != 0 s
for s in model.step:
    model.constraints.add(
        small_m * (1 - model.all_dirt_collected[s]) <= sum(model.dirt_regular_stops[i, j] for i in model.row for j in model.column) - model.b[s]
    )

# 8. El deposito solo se puede visitar dos veces
model.constraints.add(
    sum(model.r[deposit_position[0], deposit_position[1], s] for s in model.step) == 2
)

# 9. Restricción para garantizar que la segunda visita al depósito solo sea después de haber recogido toda la basura
for s in model.step:
    if s > 0:
        model.constraints.add(
            model.r[deposit_position[0], deposit_position[1], s] <= model.all_dirt_collected[s]
        )
        
# 10. Minimización del numero de pasos de vuelta al deposito
model.constraints.add(
    model.return_step == sum(s * model.r[deposit_position[0], deposit_position[1], s] for s in model.step)
)

# 11. Condición de parada. Para cada step después de haber recogido toda la basura, el camión ya no está en el mapa
for i in model.row:
    for j in model.column:
        for s in model.step:
            if s > 0:
                model.constraints.add(
                    model.r[i, j, s] <= (1 - model.r[deposit_position[0], deposit_position[1], s])  # Fuerza r[i, j, s] = 0 si hemos vuelto al depósito
                )


In [76]:
solver = po.SolverFactory('gurobi')
results = solver.solve (model, tee=True)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-14
Read LP format model from file C:\Users\sofin\AppData\Local\Temp\tmpjs9b3f_z.pyomo.lp
Reading time = 0.03 seconds
x1: 705 rows, 361 columns, 1804 nonzeros
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-8250U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 705 rows, 361 columns and 1804 nonzeros
Model fingerprint: 0x8ac348cd
Model has 17 quadratic constraints
Variable types: 18 continuous, 343 integer (342 binary)
Coefficient statistics:
  Matrix range     [1e-04, 1e+04]
  QMatrix range    [3e+00, 7e+00]
  QLMatrix range   [1e+00, 7e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 2e+01]
  RHS range        [1e+00, 1e+04]
Presolve removed 77 rows and 58 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iteratio

In [77]:
# Extraer la ruta desde model.r
ruta = []

# Iterar sobre cada camión, cada celda y cada paso (si aplica)
# for s in range(int(model.return_step.value) + 1):
for s in range(MAX_STEPS):
    for i in model.row:
        for j in model.column:
            if pe.value(model.r[i, j, s]) > 0.5:  # Comprobamos si la variable es 1
                ruta.append((i, j, s))

# Mostrar la ruta
print("Ruta del camión:")
contador = 0
for punto in ruta:
    print(f"Paso {punto[2]} : Camión en celda {punto[0], punto[1]} con una cantidad de basura: {model.b[punto[2]].value}")

ERROR: evaluating object as numeric value: r[0,0,0]
        (object: <class 'pyomo.core.base.var.VarData'>)
    No value for uninitialized NumericValue object r[0,0,0]


ValueError: No value for uninitialized NumericValue object r[0,0,0]

In [33]:
import time  

# Colores ANSI
RESET = "\033[0m"
RED = "\033[31m"  # Rojo para el paso actual
CYAN = "\033[36m"  # Cian para celdas previamente visitadas
GREEN = "\033[32m"  # Verde para celdas con basura

# Matriz con celdas vacías y cantidades de basura
matriz = [[f"{GREEN}{model.dirt_regular_stops[i, j]}{RESET}" if model.dirt_regular_stops[i, j] > 0 else " " 
           for j in range(N_COL)] 
          for i in range(N_FIL)]

print("Recorrido del camión paso a paso:\n")

# Lista para guardar las celdas visitadas
visitadas = [[False for _ in range(N_COL)] for _ in range(N_FIL)]

# Recorrido del camión paso por paso
for s in range(int(model.return_step.value) + 1):
    # Crear una copia para marcar el paso actual sin modificar la matriz original
    matriz_temp = [fila.copy() for fila in matriz]

    # Buscar la celda visitada en este paso
    for i in model.row:
        for j in model.column:
            if model.r[i, j, s].value == 1:
                matriz_temp[i][j] = f"{RED}X{RESET}"  # Marcar la celda actual en rojo
                visitadas[i][j] = True  # Marcar la celda como visitada

    # Actualizar celdas previamente visitadas a cian
    for i in range(N_FIL):
        for j in range(N_COL):
            if visitadas[i][j] and matriz_temp[i][j] != f"{RED}X{RESET}":  
                matriz_temp[i][j] = f"{CYAN}X{RESET}"

    # Mostrar la matriz actualizada
    for fila in matriz_temp:
        print("+---" * N_COL + "+")
        print("| " + " | ".join(fila) + " |")
    print("+---" * N_COL + "+\n")
    
    # Esperar antes de continuar al siguiente paso
    time.sleep(0.5) 

Recorrido del camión paso a paso:

+---+---+---+
| X |   |   |
+---+---+---+
|   |   |   |
+---+---+---+
|   | 7 | 3 |
+---+---+---+

+---+---+---+
| X |   |   |
+---+---+---+
| X |   |   |
+---+---+---+
|   | 7 | 3 |
+---+---+---+

+---+---+---+
| X |   |   |
+---+---+---+
| X | X |   |
+---+---+---+
|   | 7 | 3 |
+---+---+---+

+---+---+---+
| X |   |   |
+---+---+---+
| X | X |   |
+---+---+---+
|   | X | 3 |
+---+---+---+

+---+---+---+
| X |   |   |
+---+---+---+
| X | X |   |
+---+---+---+
|   | X | X |
+---+---+---+

+---+---+---+
| X |   |   |
+---+---+---+
| X | X | X |
+---+---+---+
|   | X | X |
+---+---+---+

+---+---+---+
| X |   |   |
+---+---+---+
| X | X | X |
+---+---+---+
|   | X | X |
+---+---+---+

+---+---+---+
| X | X |   |
+---+---+---+
| X | X | X |
+---+---+---+
|   | X | X |
+---+---+---+

+---+---+---+
| X | X |   |
+---+---+---+
| X | X | X |
+---+---+---+
|   | X | X |
+---+---+---+

